Verify that the implementation of the qDRIFT subroutine is correct. 
Compute error statistics for the qDRIFT subroutine.

In [4]:
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator
# Adjust the path to the directory containing 'scripts/algo'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts')))
from algo.qft_qpe_qdrift import *
from utils.scalable_numerical_tests import *

In [5]:
import numpy as np
import plotly.graph_objects as go

# Define parameters
times = [10, 1, 0.1, 0.01, 0.01]
for time in times:
    num_qubits = 3
    num_terms = 100
    theoretical_errors = []
    unitary_errors = []
    unitary_errors_inf = []
    diamond_distances = []
    nums_samples = [int(1.3 ** x) for x in range(0, 40)]
    # Set seaborn style
    # Generate a random Hamiltonian
    hamiltonian = generate_random_hamiltonian(num_qubits, num_terms)
    print(f"Hamiltonian: {hamiltonian}")

    # Compute exact unitary evolution
    U_exact = exact_unitary_evolution(hamiltonian, time)

    for num_samples in nums_samples:
        # Compute qDRIFT approximation
        sampled_unitaries, labels = qdrift_sample(hamiltonian, time, num_samples)
        # multiply the sampled unitaries in the order they were sampled
        U_qdrift = sampled_unitaries[0]
        for unitary in sampled_unitaries[1:]:
            U_qdrift = U_qdrift @ unitary
        U_qdrift = Operator(U_qdrift)
        # Compute errors
        error = unitary_error_2_norm(U_exact, U_qdrift)
        error_2 = unitary_error_inf_norm(U_exact, U_qdrift)
        diamond_dist = compute_diamond_distance(U_exact, U_qdrift)
        lam = sum(abs(term[0]) for term in hamiltonian)
        theoretical_error = estimate_theoretical_qdrift_errror(num_samples=num_samples, lam=lam, time=time)
        # print(f"Theoretical Error\t\tError\t\tDiamond Distance")
        # print(f"   {theoretical_error}\t    {error} \t    {diamond_dist}")
        theoretical_errors.append(2 * theoretical_error)
        unitary_errors.append(error)
        diamond_distances.append(diamond_dist)
        unitary_errors_inf.append(error_2)

    # graph error vs num sampleswith plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=nums_samples, y=theoretical_errors, mode='lines', name='Theoretical Error'))
    fig.add_trace(go.Scatter(x=nums_samples, y=unitary_errors, mode='lines', name='Unitary Error 2-norm'))
    fig.add_trace(go.Scatter(x=nums_samples, y=unitary_errors_inf, mode='lines', name='Unitary Error inf-norm'))
    fig.add_trace(go.Scatter(x=nums_samples, y=diamond_distances, mode='lines', name='Diamond Distance'))
    fig.update_layout(title=f'Error vs Number of Samples for time={time}', xaxis_title='Number of Samples', yaxis_title='Error')
    fig.show()
    

Hamiltonian: [(0.8522224081718138, Pauli('ZZX')), (0.8498980495292349, Pauli('XXX')), (0.5260304773211648, Pauli('ZII')), (0.7709264677870074, Pauli('ZIX')), (0.6135774713621182, Pauli('XII')), (0.9743140290566252, Pauli('XXZ')), (0.26802222980284574, Pauli('XZX')), (0.013522216609042403, Pauli('XZX')), (0.5248917661047888, Pauli('ZZX')), (0.9414220837550168, Pauli('IXX')), (0.0411238675487523, Pauli('ZXI')), (0.876009037357485, Pauli('IXI')), (0.712726591598201, Pauli('IIX')), (0.11248552608660911, Pauli('IZX')), (0.06656104235262117, Pauli('XXZ')), (0.8221180235528504, Pauli('XII')), (0.6669874881789014, Pauli('XII')), (0.9535435700873219, Pauli('IIZ')), (0.291054963956794, Pauli('III')), (0.9710698638631035, Pauli('ZIZ')), (0.06673243015036745, Pauli('ZZI')), (0.9040405169009786, Pauli('IIX')), (0.7236968772821264, Pauli('IZI')), (0.9868376112475606, Pauli('XIX')), (0.5740761533891625, Pauli('ZXX')), (0.29436563481732203, Pauli('XXX')), (0.7799759831317278, Pauli('IXX')), (0.2545946

c:\Users\rodcs\Desktop\my-repos\qDriftExtraPolation\scripts\algo\qft_qpe_qdrift.py:50: RuntimeWarning: overflow encountered in exp
  return (2 * lam**2 * time**2 / num_samples) * np.exp(2 * lam * time / num_samples)


Hamiltonian: [(0.27235404127007756, Pauli('ZZZ')), (0.409634208112841, Pauli('ZZZ')), (0.654510566735098, Pauli('IZZ')), (0.5507347045828066, Pauli('IZZ')), (0.016228659330432604, Pauli('IZX')), (0.7750995472087142, Pauli('IIZ')), (0.8679997437035516, Pauli('ZXI')), (0.9101402616466958, Pauli('ZIX')), (0.8334024337347204, Pauli('IZX')), (0.7592630645749673, Pauli('IZX')), (0.08166783695203061, Pauli('IIX')), (0.6755920171138291, Pauli('IIX')), (0.06381280319079319, Pauli('XZX')), (0.9473988522591325, Pauli('IXI')), (0.21714181220720086, Pauli('ZXZ')), (0.7127122431641795, Pauli('ZXX')), (0.008328194487258855, Pauli('XXX')), (0.8196072449616889, Pauli('IXZ')), (0.23667546656777205, Pauli('III')), (0.4674631215733829, Pauli('ZIZ')), (0.3271767383112626, Pauli('XXI')), (0.8730287146743669, Pauli('XXX')), (0.13889236520574377, Pauli('ZIZ')), (0.1657963239442637, Pauli('IIZ')), (0.6183107553369286, Pauli('IZX')), (0.9231825897632719, Pauli('IXX')), (0.15174816924930778, Pauli('ZXZ')), (0.60

Hamiltonian: [(0.03205437696196811, Pauli('ZII')), (0.5743848937995297, Pauli('XII')), (0.30385611372079235, Pauli('ZIZ')), (0.6980982061807371, Pauli('ZXZ')), (0.9369331903666472, Pauli('IXX')), (0.5246770154794687, Pauli('XZI')), (0.14788880307738494, Pauli('IXI')), (0.38134818721902297, Pauli('IXZ')), (0.8450998428369857, Pauli('XZZ')), (0.6533677679209039, Pauli('XZX')), (0.4952882443654534, Pauli('IIX')), (0.07854764887642673, Pauli('IZX')), (0.8087692819342339, Pauli('XIX')), (0.49360278617308384, Pauli('IXX')), (0.9616118080991094, Pauli('XZI')), (0.8684038678843451, Pauli('III')), (0.9042539762243841, Pauli('IZZ')), (0.740546719895961, Pauli('ZXX')), (0.0526826783652119, Pauli('XXX')), (0.8223340453362072, Pauli('ZXZ')), (0.7296816205701935, Pauli('IXX')), (0.3207541682312032, Pauli('IIZ')), (0.286429730886662, Pauli('ZIX')), (0.5868231911903328, Pauli('IZX')), (0.2938803123659144, Pauli('ZZI')), (0.7727618467042071, Pauli('XXI')), (0.11611950999258336, Pauli('ZIX')), (0.709089

Hamiltonian: [(0.29791615704243757, Pauli('ZZI')), (0.8944998804995491, Pauli('IZI')), (0.4781115282661804, Pauli('ZIX')), (0.0681712612325518, Pauli('ZZI')), (0.8019968318999094, Pauli('XIX')), (0.3359755710820792, Pauli('ZZZ')), (0.8860995143431604, Pauli('ZIZ')), (0.04870973904069609, Pauli('ZIX')), (0.21047859879514608, Pauli('IXZ')), (0.00616176021567294, Pauli('IXZ')), (0.8066465186921069, Pauli('XIZ')), (0.916877547232687, Pauli('IZI')), (0.6383203283704961, Pauli('ZXX')), (0.34624210423994006, Pauli('IXI')), (0.48019434803746897, Pauli('ZXX')), (0.09315597055960967, Pauli('IXX')), (0.1007833456142565, Pauli('IIZ')), (0.6253659428772028, Pauli('ZIZ')), (0.7428231835228813, Pauli('XZX')), (0.9956270655642934, Pauli('XXI')), (0.09444741173281823, Pauli('ZIZ')), (0.6852291425120092, Pauli('ZXI')), (0.07848464389347432, Pauli('IXX')), (0.6113566745416555, Pauli('XZI')), (0.20455791195831852, Pauli('ZXX')), (0.22789878429822152, Pauli('ZIX')), (0.9073373368967065, Pauli('XXX')), (0.4

Hamiltonian: [(0.6618261988488137, Pauli('XZI')), (0.4044811757495229, Pauli('ZXZ')), (0.22823534459193484, Pauli('ZII')), (0.37810427411256087, Pauli('XIZ')), (0.6142505087748484, Pauli('ZIZ')), (0.9097569917043744, Pauli('ZZZ')), (0.6373033430036187, Pauli('IIZ')), (0.7358155799654738, Pauli('IZX')), (0.15662482870962302, Pauli('ZXX')), (0.9056649909227075, Pauli('ZZX')), (0.39442410612008294, Pauli('XIX')), (0.728803867158402, Pauli('III')), (0.24679072135339952, Pauli('IXZ')), (0.7222537314061178, Pauli('XXZ')), (0.8017882363751021, Pauli('XZI')), (0.11790240577811761, Pauli('ZXX')), (0.6153649687765658, Pauli('ZII')), (0.6346135937862478, Pauli('ZIZ')), (0.09979589630552377, Pauli('ZXZ')), (0.7370139065055887, Pauli('IZX')), (0.9008464288948764, Pauli('ZZZ')), (0.10238335088353057, Pauli('XII')), (0.7880495489905059, Pauli('XXZ')), (0.8588517237608374, Pauli('XZZ')), (0.05314751027438491, Pauli('IIX')), (0.8838829920091216, Pauli('IZI')), (0.274782033750234, Pauli('XXZ')), (0.5551

In [6]:
# Lets try to calculate the number of samples needed to achieve a certain error
# Define parameters
times = [10, 1, 0.1, 0.01, 0.001]
target_epsilon = 0.01
num_qubits = 2
num_terms = 100
hamiltonian = generate_random_hamiltonian(num_qubits, num_terms)
lam = sum(abs(term[0]) for term in hamiltonian)
theoretical_errors_a_priori = []
theoretical_errors_a_posteriori = []
unitary_errors = []
diamond_distances = []
print(f"Hamiltonian: {hamiltonian}")
for time in times:
    ideal_num_samples = 2 * lam ** 2 * time ** 2 / target_epsilon
    print(f"Ideal number of samples: {ideal_num_samples}")
    # Compute exact unitary evolution
    U_exact = exact_unitary_evolution(hamiltonian, time)
    sampled_unitaries, labels = qdrift_sample(hamiltonian, time, int(ideal_num_samples))
    # multiply the sampled unitaries in the order they were sampled
    U_qdrift = sampled_unitaries[0]
    for unitary in sampled_unitaries[1:]:
        U_qdrift = U_qdrift @ unitary
    U_qdrift = Operator(U_qdrift)
    # Compute errors
    error = unitary_error(U_exact, U_qdrift)
    diamond_dist = compute_diamond_distance(U_exact, U_qdrift)
    theoretical_error = estimate_theoretical_qdrift_errror(num_samples=int(ideal_num_samples), lam=lam, time=time)
    print(f"Theoretical Error\tError\tDiamond Distance")
    print(f"   {theoretical_error}\t    {error} \t    {diamond_dist}")
    theoretical_errors_a_priori.append(target_epsilon)
    theoretical_errors_a_posteriori.append(theoretical_error)
    unitary_errors.append(error)
    diamond_distances.append(diamond_dist)

# graph error vs num sampleswith seaborn
plt.plot(times, theoretical_errors_a_priori, label="Target Error")
plt.plot(times, theoretical_errors_a_posteriori, label="Theoretical Error")
plt.plot(times, unitary_errors, label="Unitary Error")
plt.plot(times, diamond_distances, label="Diamond Distance")

plt.xlabel("Time")
plt.ylabel("Error")
plt.legend()
plt.show()


Hamiltonian: [(0.6644737426080758, Pauli('XX')), (0.5546315621076533, Pauli('XZ')), (0.685248696401256, Pauli('IX')), (0.7356002955436223, Pauli('II')), (0.3999299230113821, Pauli('ZI')), (0.5232708041178867, Pauli('IX')), (0.629991514944065, Pauli('XI')), (0.790621377058288, Pauli('ZZ')), (0.38376181858514524, Pauli('II')), (0.2332551368005643, Pauli('IX')), (0.10623645535034332, Pauli('XZ')), (0.5745793447155367, Pauli('XX')), (0.377994362278278, Pauli('IX')), (0.6176589885460047, Pauli('IX')), (0.40775672386748396, Pauli('ZZ')), (0.3900379664273893, Pauli('ZX')), (0.746882047326055, Pauli('ZZ')), (0.8922668349088267, Pauli('XX')), (0.3245830916867084, Pauli('XI')), (0.11945548232156211, Pauli('II')), (0.30026953127654155, Pauli('ZX')), (0.6708483896629269, Pauli('II')), (0.5886747936665737, Pauli('XX')), (0.5218228184551863, Pauli('XX')), (0.05111208051906169, Pauli('ZX')), (0.6002636979490329, Pauli('II')), (0.48114560864271716, Pauli('XX')), (0.4790069646294668, Pauli('ZZ')), (0.6

NameError: name 'unitary_error' is not defined